In [80]:
pip install nltk

In [81]:
pip install spacy

In [82]:
#import libaries
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
import numpy as np
import pandas as pd
import sklearn
import spacy

Import text data

In [83]:
#I am using CNN articles for the analysis
#It has been already downloaded to my google drive
PATH = '/content/drive/MyDrive/NLP_text/cnn_lite'

DOC_PATTERN = r'.*\.txt'
corpus = PlaintextCorpusReader(PATH, DOC_PATTERN)

In [99]:
#separate articles with corpus based on their fileID
articles = [corpus.raw(fileid) for fileid in corpus.fileids()]

First, make sure to clean your data. You can use the data-cleaning techniques that you practiced in the Text preprocessing checkpoint. But keep in mind that every text dataset is likely to have its own particular cleaning requirements. Make sure to clean your dataset appropriately.

In [85]:
#I use the text_cleaner function to clean the articles
def text_cleaner(text):
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

In [86]:
# Load and clean the data
art_1 = docs[0]
art_2 = docs[1]
art_3 = docs[2]
art_4 = docs[3]
art_5 = docs[4]

art_1 = text_cleaner(art_1)
art_2 = text_cleaner(art_2)
art_3 = text_cleaner(art_3)
art_4 = text_cleaner(art_4)
art_5 = text_cleaner(art_5)

In [87]:
# Parse the cleaned articles
nlp = spacy.load('en')
art_1_doc = nlp(art_1)
art_2_doc = nlp(art_2)
art_3_doc = nlp(art_3)
art_4_doc = nlp(art_4)
art_5_doc = nlp(art_5)

In [88]:
# Group into sentences
art1_sent = [[sent, "Article1"] for sent in art_1_doc.sents]
art2_sent = [[sent, "Article2"] for sent in art_2_doc.sents]
art3_sent = [[sent, "Article3"] for sent in art_3_doc.sents]
art4_sent = [[sent, "Article4"] for sent in art_4_doc.sents]
art5_sent = [[sent, "Article5"] for sent in art_5_doc.sents]


# Combine the sentences from the 5 articles into one DataFrame
sentences = pd.DataFrame(art1_sent + art2_sent + art3_sent + art4_sent + art5_sent, columns = ["text", "article_number"])
sentences.tail()

,text,article_number
458,"(©, Cable, News, Network, .)",Article5
459,"(Turner, Broadcasting, System, ,)",Article5
460,"(Inc., All, Rights, Reserved, .)",Article5
461,"(Listen, to, CNN, (, low, -, bandwidth, usage,...",Article5
462,"(Go, to, the, full, CNN, experience, .)",Article5


In [89]:
# Get rid of stop words and punctuation,
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = " ".join(
        [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])

Convert your text features into numerical form. You're free to choose your method. You can use BoW, TF-IDF, word2vec, or any other method that you'd like to experiment with. It's a good idea to use several methods so that you can compare the results with respect to the methods that you apply.

In [95]:
#First I will use CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word')
X = vectorizer.fit_transform(sentences["text"])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([bow_df, sentences[["text", "article_number"]]], axis=1)

In [96]:
sentences.head()

,1990s,23andme,30,35s,40,4th,aback,ability,able,absolutely,absorption,academic,accept,access,accord,acidly,acknowledge,acquisition,act,action,active,actually,adam,adams,add,address,adjust,administration,adultery,adviser,affair,afford,aftermath,age,aged,aggressive,ago,agony,agree,aide,...,wave,way,weapon,wednesday,week,welcome,welfare,well,whisper,white,widen,wife,will,willing,win,winner,wish,witch,witness,woman,wonder,word,work,working,workplace,world,worry,worsen,write,wrong,wry,year,yell,yes,york,young,youngster,zone,text,article_number
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Des Moines Iowa CNN,Article1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rep. Alexandria Ocasio Cortez cap debut Iowa h...,Article1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,allow happen,Article1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,let race happen say rally Vermont senator Coun...,Article1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,watch presidential race,Article1


In [97]:
#Now I will also use TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True)


# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences_tfidf = pd.concat([tfidf_df, sentences[["text", "article_number"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,1990s,23andme,30,35s,40,4th,aback,ability,able,absolutely,absorption,academic,accept,access,accord,acidly,acknowledge,acquisition,act,action,active,actually,adam,adams,add,address,adjust,administration,adultery,adviser,affair,afford,aftermath,age,aged,aggressive,ago,agony,agree,aide,...,wave,way,weapon,wednesday,week,welcome,welfare,well,whisper,white,widen,wife,will,willing,win,winner,wish,witch,witness,woman,wonder,word,work,working,workplace,world,worry,worsen,write,wrong,wry,year,yell,yes,york,young,youngster,zone,text,article_number
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Des Moines Iowa CNN,Article1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rep. Alexandria Ocasio Cortez cap debut Iowa h...,Article1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,allow happen,Article1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,let race happen say rally Vermont senator Coun...,Article1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,watch presidential race,Article1


You can choose whatever NLP task you want. You're completely free in your choice. Here are some possible options:
Sentiment analysis
Text classification
Topic modeling
Developing a more sophisticated chatbot than what you've already developed in this module
Training a deep-learning model for a supervised or unsupervised task of your choice

In [98]:
#I am going to conduct Text Classfication
#First, I use the CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Y = sentences['article_number']
X = np.array(sentences.drop(['text','article_number'], 1))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.9061371841155235

Test set score: 0.7365591397849462
----------------------Random Forest Scores----------------------
Training set score: 0.9314079422382672

Test set score: 0.7526881720430108
----------------------Gradient Boosting Scores----------------------
Training set score: 0.9314079422382672

Test set score: 0.7580645161290323


In [94]:
#Now I will use the TF-IDF
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Y = sentences_tfidf['article_number']
X = np.array(sentences.drop(['text','article_number'], 1))

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.9061371841155235

Test set score: 0.7365591397849462
----------------------Random Forest Scores----------------------
Training set score: 0.9314079422382672

Test set score: 0.7634408602150538
----------------------Gradient Boosting Scores----------------------
Training set score: 0.927797833935018

Test set score: 0.7580645161290323


Overall, CountVectorizer prepared the data better for most of the models, but Gradient boosting has a slightly less overfitting in TF-IDF